## Установим пайспарк и созадим спарк-сессию

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
!mkdir sample_data/accounts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 55.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d9cdaef4e4c20ee9c31fa26acea1720d18b75ae831b4b320fe607055e926b4ff
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


## Создадим json-файлы с данными о клиентах банка

In [2]:
!echo '{  "id": "a1globalid",  "op": "c",  "ts": 1577863800000,  "data": {    "account_id": "a1",    "name": "Anton",    "address": "Moscow",    "phone_number": "12345678",    "email": "anthony@somebank.com"  }}' > /content/sample_data/accounts/1577863800000.json
!echo '{  "id": "a2globalid",  "op": "c",  "ts": 1577873800000,  "data": {    "account_id": "a2",    "name": "Alex",    "address": "Atlanta",    "phone_number": "98765432",    "email": "Alex@somebank.com"  }}' > /content/sample_data/accounts/1577873800000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1577865600000,  "set": {    "phone_number": "87654321"  }}' > /content/sample_data/accounts/1577865600000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1577890800000,  "set": {    "savings_account_id": "sa1"  }}' > /content/sample_data/accounts/1577890800000.json
!echo '{  "id": "a2globalid",  "op": "u",  "ts": 1577890900000,  "set": {    "savings_account_id": "sa2"  }}' > /content/sample_data/accounts/1577890900000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1577894400000,  "set": {    "address": "Yekaterinburg",    "email": "anthony@anotherbank.com"  }}' > /content/sample_data/accounts/1577894400000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1577926800000,  "set": {    "card_id": "c1"  }}' > /content/sample_data/accounts/1577926800000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1579078860000,  "set": {    "card_id": ""  }}' > /content/sample_data/accounts/1579078860000.json
!echo '{  "id": "a1globalid",  "op": "u",  "ts": 1579163400000,  "set": {    "card_id": "c2"  }}' > /content/sample_data/accounts/1579163400000.json

In [36]:
from pyspark.sql import *
from pyspark.sql.functions import *
import timeit

## Прочитаем json-файлы и создадим датафрейм

In [26]:
df = spark.read.json('/content/sample_data/accounts').orderBy(col('id'), col('ts'))
df.show()
df.printSchema()

+--------------------+----------+---+--------------------+-------------+
|                data|        id| op|                 set|           ts|
+--------------------+----------+---+--------------------+-------------+
|{a1, Moscow, anth...|a1globalid|  c|                null|1577863800000|
|                null|a1globalid|  u|{null, null, null...|1577865600000|
|                null|a1globalid|  u|{null, null, null...|1577890800000|
|                null|a1globalid|  u|{Yekaterinburg, n...|1577894400000|
|                null|a1globalid|  u|{null, c1, null, ...|1577926800000|
|                null|a1globalid|  u|{null, , null, nu...|1579078860000|
|                null|a1globalid|  u|{null, c2, null, ...|1579163400000|
|{a2, Atlanta, Ale...|a2globalid|  c|                null|1577873800000|
|                null|a2globalid|  u|{null, null, null...|1577890900000|
+--------------------+----------+---+--------------------+-------------+

root
 |-- data: struct (nullable = true)
 |    |--

## Приведём датафрейм к нормальному виду без вложенностей

In [27]:
df = df.select(
    'ts', 
    split(col('id'), 'globalid')
      .getItem(0).alias('account_id'),

    coalesce(
      col('data')['address'],
      col('set')['address']
    ).alias('address'),

    coalesce(
      col('data')['email'],
      col('set')['email']
    ).alias('email'),
    
    col('data')['name'].alias('name'),
    
    coalesce(
      col('data')['phone_number'],
      col('set')['phone_number']
    ).alias('phone_number'),
    
    col('set')['card_id'].alias('card_id'),
    col('set')['savings_account_id'].alias('savings_account_id')
).orderBy(col('account_id'), col('ts'))

df.show()

+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|           ts|account_id|      address|               email| name|phone_number|card_id|savings_account_id|
+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|1577863800000|        a1|       Moscow|anthony@somebank.com|Anton|    12345678|   null|              null|
|1577865600000|        a1|         null|                null| null|    87654321|   null|              null|
|1577890800000|        a1|         null|                null| null|        null|   null|               sa1|
|1577894400000|        a1|Yekaterinburg|anthony@anotherba...| null|        null|   null|              null|
|1577926800000|        a1|         null|                null| null|        null|     c1|              null|
|1579078860000|        a1|         null|                null| null|        null|       |              null|
|1579163400000|        a1|  

## Правильный метод

Используем оконную функцию и функцию last:

In [37]:
from pyspark.sql.window import Window

win = Window().partitionBy('account_id').orderBy('ts')
start = timeit.default_timer()
df2 = (df.select(
      'ts', 
      'account_id', 
      last('address', True).over(win).alias('address'),
      last('email', True).over(win).alias('email'),
      last('name', True).over(win).alias('name'),
      last('phone_number', True).over(win).alias('phone_number'),
      last('card_id', False).over(win).alias('card_id'),
      last('savings_account_id', True).over(win).alias('savings_account_id')
    )
)
stop = timeit.default_timer()
df2.show()

+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|           ts|account_id|      address|               email| name|phone_number|card_id|savings_account_id|
+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|1577863800000|        a1|       Moscow|anthony@somebank.com|Anton|    12345678|   null|              null|
|1577865600000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|              null|
|1577890800000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|               sa1|
|1577894400000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|   null|               sa1|
|1577926800000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|     c1|               sa1|
|1579078860000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|       |               sa1|
|1579163400000|        a1|Ye

In [38]:
stop-start

0.05409697300001426

Результат намного лучше тех, что представлены ниже :)

## Метод №1: хард-код

Создадим функцию pullData, которая принимает в себя списки и заполняет пропуски там, где их не должно быть: 

In [29]:
def pullData(*args):
  for col in args:
    val = None
    for i, _ in enumerate(range(len(col))):
      if col[i] != None:
        val = col[i]
      col[i] = val

Для каждого столбца датафрейма создадим свой список, в который записываются данные с помощью генератора списков. Затем применим к ним вышеописанную функцию pullData. Затем, нам нужно вложить их в другой список (также генератором) и транспонировать его. Затем на основе этого транспонированного списка и схемы оригинального датафрейма создадим новый датафрейм. Для понятия производительности метода, засечём время выполнения с помощью модуля timeit.

In [30]:
start = timeit.default_timer()

ts = [row[0] for row in df.select('ts').collect()]
account_id = [row[0] for row in df.select('account_id').collect()]
address = [row[0] for row in df.select('address').collect()]
email = [row[0] for row in df.select('email').collect()]
name = [row[0] for row in df.select('name').collect()]
phone_number = [row[0] for row in df.select('phone_number').collect()]
card_id = [row[0] for row in df.select('card_id').collect()]
savings_account_id = [row[0] for row in df.select('savings_account_id').collect()]

pullData(ts, account_id, address, email, name, phone_number, card_id, savings_account_id)

a = [[ts[i], account_id[i], address[i], email[i], name[i], phone_number[i], card_id[i], savings_account_id[i]] for i in range(df.count())]

stop = timeit.default_timer()

spark.createDataFrame(a, df.columns).show()

+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|           ts|account_id|      address|               email| name|phone_number|card_id|savings_account_id|
+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|1577863800000|        a1|       Moscow|anthony@somebank.com|Anton|    12345678|   null|              null|
|1577865600000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|              null|
|1577890800000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|               sa1|
|1577894400000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|   null|               sa1|
|1577926800000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|     c1|               sa1|
|1579078860000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|       |               sa1|
|1579163400000|        a1|Ye

На малом количестве данных данный способ мы имеем следующее время выполнения:

In [31]:
stop-start

4.586826756999926

С увеличением количества данных эта цифра будет необратимо расти и придётся искать более адаптированный инструмент для обработки.
(способа сделать это pyspark'ом я не нашёл)

## Метод №2: функциональное программирование

Создадим функцию dfColumnsToListOfLists, которая будет создавать список списков данных из колонок датафрейма:

In [32]:
def dfColumnsToListOfLists(df: DataFrame) -> list:
  cols = df.columns
  res = []
  for col in cols:
    res += [[row[0] for row in df.select(col).collect()]]
  return res

Далее, создадим функцию pullData2dList, которая будет изменять данные в списке, которую возвращает прошлая функция (алгоритм практически такой же, как и в методе №1):

In [33]:
def pullData2dList(columnsList: list) -> list:
  resList = []
  for colName in columnsList:
    val = None
    for i, _ in enumerate(colName):
      if colName[i] != None:
        val = colName[i]
      colName[i] = val
    resList += [colName]
  return resList

Затем создадим функцию, которая транспонирует наш список:

In [34]:
def transposeLists(lists: list) -> list:
  return [list(i) for i in zip(*lists)]

Проверим работоспособность этого алгоритма:

In [35]:
import timeit

start = timeit.default_timer()

df2 = spark.createDataFrame(transposeLists(pullData2dList(dfColumnsToListOfLists(df))), df.columns)

stop = timeit.default_timer()
  
df2.show()

+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|           ts|account_id|      address|               email| name|phone_number|card_id|savings_account_id|
+-------------+----------+-------------+--------------------+-----+------------+-------+------------------+
|1577863800000|        a1|       Moscow|anthony@somebank.com|Anton|    12345678|   null|              null|
|1577865600000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|              null|
|1577890800000|        a1|       Moscow|anthony@somebank.com|Anton|    87654321|   null|               sa1|
|1577894400000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|   null|               sa1|
|1577926800000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|     c1|               sa1|
|1579078860000|        a1|Yekaterinburg|anthony@anotherba...|Anton|    87654321|       |               sa1|
|1579163400000|        a1|Ye

In [ ]:
stop-start

2.5537970740000446

Цифра немногим лучше, чем прошлый метод.